<a href="https://colab.research.google.com/github/xoxominji/22-1-ESAA-Practice/blob/main/0523_session_%EB%B0%95%EB%AF%BC%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Reviews를 이용한 추천시스템 실습 - surprise패키지 사용(36~)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
electronics_data = pd.read_csv("/content/drive/MyDrive/2022-1 ESAA/데이터/ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])

In [ ]:
# 평가가 50번 이상 된 상품들만 추출
new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

## **Collaborative Filtering 이용**

In [ ]:
pip install surprise

     |████████████████████████████████| 11.8 MB 7.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633688 sha256=588bcd0cfcde9a29fa07050a117626dca5c9487185d2893e45eeb6b572f3bfd4
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

### 1) Data 가져오기

In [ ]:
new_df.drop(['timestamp'],axis = 1, inplace = True)

In [ ]:
reader = Reader(rating_scale=(1,5)) #단위 1, 최대 5
data = Dataset.load_from_df(new_df, reader)

### 2) Trainset, Testset으로 나누기

In [ ]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

### 3) KNN을 이용 trainset에 파라미터 맞추기

In [ ]:
algo = KNNWithMeans(k=5, sim_options = {'name': 'pearson_baseline', 'user_based':False})
algo.fit(trainset)

In [ ]:
algo = KNNWithMeans(k = 5, sim_options = {'name': 'pearson_baseline', 'user_based':False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


### 4) Test accuracy 측정

In [ ]:
test_pred = algo.test(testset)

In [ ]:
print('Item-based Model : Test Set')
accuracy.rmse(test_pred, verbose = True)

Item-based Model : Test Set
RMSE: 1.3386


1.3385712631956137

## Model-based collaborative filtering system (모델 기반 협업 필터링)

In [ ]:
# 대량의 상품, 많은 사용자들에게 추천 가능, sparse한 matrices에도 사용가능

In [ ]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

productId,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,1400532736,1400599997,1400698987,...,B00000JFMK,B00000JHWX,B00000JI4F,B00000JII6,B00000JMUG,B00000JPPI,B00000JSGF,B00000JYLO,B00000JYWQ,B00000K135
userId,,,,,,,,,,,,,,,,,,,,,
A01852072Z7B68UHLI5UG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0266076X6KPZ6CCHGVS,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0293130VTX2ZXA70JQS,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A030530627MK66BD8V4LN,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0571176384K8RBNKGF8O,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings_matrix.shape

(9832, 76)

In [ ]:
X = ratings_matrix.T
X.head()

userId,A01852072Z7B68UHLI5UG,A0266076X6KPZ6CCHGVS,A0293130VTX2ZXA70JQS,A030530627MK66BD8V4LN,A0571176384K8RBNKGF8O,A0590501PZ7HOWJKBGQ4,A0641581307AKT5MAOU0Q,A076219533YHEV2LJO988,A0821988FXKFYX53V4QG,A099626739FNCRNHIKBCG,...,AZWOPBY75SGAM,AZX0ZDVAFMN78,AZX5LAN9JEAFF,AZX7I110AF0W2,AZXKUK895VGSM,AZXP46IB63PU8,AZYTSU42BZ7TP,AZZGJ2KMWB7R,AZZMV5VT9W7Y8,AZZST8OYL5P4Q
productId,,,,,,,,,,,,,,,,,,,,,
0972683275,0,0,5,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501466,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1400501520,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1400501776,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1400532620,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X.shape

(76, 9832)

In [ ]:
X.index #product ID

Index(['0972683275', '1400501466', '1400501520', '1400501776', '1400532620',
       '1400532655', '140053271X', '1400532736', '1400599997', '1400698987',
       '3744295508', '6301977173', '7214047977', '8862935293', '9573212919',
       '9575871979', '9625993428', '9888002198', '9966694544', '9983891212',
       '9984984354', '9985511476', 'B000001OM4', 'B000001OM5', 'B000001OMI',
       'B000001ON6', 'B00000DM9W', 'B00000IGBF', 'B00000J05A', 'B00000J061',
       'B00000J08Q', 'B00000J0D2', 'B00000J0D5', 'B00000J0D8', 'B00000J1EJ',
       'B00000J1EP', 'B00000J1EQ', 'B00000J1F3', 'B00000J1GA', 'B00000J1QK',
       'B00000J1QR', 'B00000J1SC', 'B00000J1TX', 'B00000J1U8', 'B00000J1UQ',
       'B00000J1V3', 'B00000J1V5', 'B00000J3NF', 'B00000J3Q7', 'B00000J3UJ',
       'B00000J434', 'B00000J4EY', 'B00000J4FS', 'B00000J4GE', 'B00000J6WY',
       'B00000JBAT', 'B00000JBHP', 'B00000JBPB', 'B00000JCT8', 'B00000JCTO',
       'B00000JD34', 'B00000JDF5', 'B00000JDF6', 'B00000JDHV', 'B00000JFE3',

In [ ]:
X1 = X

In [ ]:
from sklearn.decomposition import TruncatedSVD 
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(76, 10)

> Truncated SVD는 ∑의 대각 원소 중 상위 몇 개만 추출하고 여기에 대응하는 U와 V의 원소도 함께 제거해 차원을 줄인 것

In [ ]:
decomposed_matrix #차원 축소된 행렬

array([[ 1.48104508e+02, -1.54647509e-01,  2.45173275e-02,
         4.41034474e-05, -1.34192251e-01, -5.15620995e-06,
         4.61244041e-02, -5.01481315e-04,  2.57435335e-03,
        -8.02459302e-02],
       [ 6.70213005e-04,  1.28730971e+00,  1.48003190e+00,
         2.00382938e-01,  6.32146414e-04, -1.05683062e-03,
         8.76003073e-03,  6.57658611e-02,  6.14718360e+01,
         7.26930783e-01],
       [ 4.49410877e-05,  9.53330278e-02,  3.14825601e-01,
         5.96994838e-03, -6.60895803e-03,  1.77939588e-03,
         1.50013536e-02,  1.22913029e-02,  1.52374206e+00,
        -2.77054619e-01],
       [ 5.67764984e-04,  7.69362088e-01,  4.98361142e-01,
         1.66133138e-02,  1.06064423e-02,  1.00733163e-02,
        -5.82029372e-02,  6.02341575e-02,  4.48061508e+00,
        -5.20938427e-02],
       [ 4.66645763e-04,  7.95687332e-01,  1.05372861e+00,
        -2.17052137e-03, -2.68788807e-02, -4.67343110e-02,
         5.25195180e-02, -2.33115968e-01, -5.37324227e-01,
         8.

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(76, 76)

In [ ]:
X.index[75]

'B00000K135'

In [ ]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i) #index구하기
product_ID

75

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(76,)

In [ ]:
correlation_product_ID

array([-0.10233945, -0.24216919, -0.43999116, -0.28810319,  0.42886399,
       -0.05725723, -0.15565915, -0.14508945, -0.80399319, -0.30659782,
       -0.68615325, -0.34584928,  0.58148823,  0.51501301,  0.26216567,
       -0.38979736, -0.54211551, -0.57931797,  0.73610553,  0.95089546,
        0.71980151, -0.1824852 ,  0.90406504, -0.84354801,  0.17080679,
        0.7330074 , -0.12304221,  0.18302771,  0.72364552,  0.17996943,
        0.96634251,  0.82376286,  0.92040494, -0.81617631,  0.81839061,
        0.7774342 , -0.7977821 ,  0.58657258, -0.06435138, -0.86001175,
       -0.24465357,  0.69023108,  0.15530716, -0.22962574, -0.05214684,
        0.3503531 , -0.09616001, -0.67941491,  0.10594832,  0.77168856,
        0.75557152,  0.51575275,  0.67921687, -0.28316693, -0.0711231 ,
        0.92735259, -0.8473428 ,  0.22886298,  0.08517713, -0.23021316,
        0.87725695, -0.21466845,  0.10908302,  0.17903405, -0.95537502,
        0.8367673 ,  0.31793042,  0.83962064, -0.12232849,  0.86

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.65]) #피어슨 상관계수가 0.65보다 큰 상품들의 index의 list return

Recommend.remove(i) # 이미 구입한 상품은 뺀다

Recommend[0:24] #상위 24개 추천

['9966694544',
 '9983891212',
 '9984984354',
 'B000001OM4',
 'B000001ON6',
 'B00000J05A',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J0D5',
 'B00000J1EJ',
 'B00000J1EP',
 'B00000J1SC',
 'B00000J3UJ',
 'B00000J434',
 'B00000J4FS',
 'B00000JBAT',
 'B00000JD34',
 'B00000JFIF',
 'B00000JHWX',
 'B00000JII6',
 'B00000JMUG',
 'B00000JPPI',
 'B00000JYWQ']

In [ ]:
Recommend

['9966694544',
 '9983891212',
 '9984984354',
 'B000001OM4',
 'B000001ON6',
 'B00000J05A',
 'B00000J08Q',
 'B00000J0D2',
 'B00000J0D5',
 'B00000J1EJ',
 'B00000J1EP',
 'B00000J1SC',
 'B00000J3UJ',
 'B00000J434',
 'B00000J4FS',
 'B00000JBAT',
 'B00000JD34',
 'B00000JFIF',
 'B00000JHWX',
 'B00000JII6',
 'B00000JMUG',
 'B00000JPPI',
 'B00000JYWQ']